In [1]:
import numpy as np
import pandas as pd
import tools
import tca
import sklearn
import random
from numpy.random import seed
from sklearn.metrics import accuracy_score
import warnings

warnings.filterwarnings('ignore')

# A = pd.read_csv("/cs/home/ms383/Dissertation/data/A.csv", header=None) 
# B = pd.read_csv("/cs/home/ms383/Dissertation/data/B.csv", header=None) 
# C = pd.read_csv("/cs/home/ms383/Dissertation/data/C.csv", header=None) 
# AB_sim = pd.read_csv("/cs/home/ms383/Dissertation/data/Sensor_Sim_AB.csv", header=None, sep = '\t')
# BC_sim = pd.read_csv("/cs/home/ms383/Dissertation/data/Sensor_Sim_BC.csv", header=None, sep = '\t')
# AC_sim = pd.read_csv("/cs/home/ms383/Dissertation/data/Sensor_Sim_AC.csv", header=None, sep = '\t')

A = pd.read_csv("D:/workspace/jupyter/Dissertation/data/A.csv", header=None) 
B = pd.read_csv("D:/workspace/jupyter/Dissertation/data/B.csv", header=None) 
C = pd.read_csv("D:/workspace/jupyter/Dissertation/data/C.csv", header=None) 
AB_sim = pd.read_csv("D:/workspace/jupyter/Dissertation/data/Sensor_Sim_AB.csv", header=None, sep = '\t')
BC_sim = pd.read_csv("D:/workspace/jupyter/Dissertation/data/Sensor_Sim_BC.csv", header=None, sep = '\t')
AC_sim = pd.read_csv("D:/workspace/jupyter/Dissertation/data/Sensor_Sim_AC.csv", header=None, sep = '\t')

# A = pd.read_csv("/Users/fred/Desktop/Dissertation/Dissertation/data/A.csv", header=None) 
# B = pd.read_csv("/Users/fred/Desktop/Dissertation/Dissertation/data/B.csv", header=None) 
# C = pd.read_csv("/Users/fred/Desktop/Dissertation/Dissertation/data/C.csv", header=None) 
# AB_sim = pd.read_csv("/Users/fred/Desktop/Dissertation/Dissertation/data/Sensor_Sim_AB.csv", header=None, sep = '\t')
# BC_sim = pd.read_csv("/Users/fred/Desktop/Dissertation/Dissertation/data/Sensor_Sim_BC.csv", header=None, sep = '\t')
# AC_sim = pd.read_csv("/Users/fred/Desktop/Dissertation/Dissertation/data/Sensor_Sim_AC.csv", header=None, sep = '\t')

x_a = np.array(A.values[:, 1:A.shape[1]])
y_a = np.array(np.transpose([A.values[:, 0]]))
x_b = np.array(B.values[:, 1:B.shape[1]])
y_b = np.array(np.transpose([B.values[:, 0]]))
x_c = np.array(C.values[:, 1:C.shape[1]])
y_c = np.array(np.transpose([C.values[:, 0]]))

Ab = np.dot(x_a,AB_sim)
Ac = np.dot(x_a,AC_sim)
Ba = np.dot(x_b,AB_sim.T)
Bc = np.dot(x_b,BC_sim)
Ca = np.dot(x_c,AC_sim.T)
Cb = np.dot(x_c,BC_sim.T)

In [3]:
options = {'dim':20,
           'kerneltype':'rbf',
           'kernelparam':1.0,
           'mu':1.0}

In [104]:
def runhtca(x_src,y_src,x_tar,y_tar):
    ################################################################# prepare
    tf='TCA'
    label_set = np.unique(y_src).astype(int)

    #classifiers
    ##RF
    from sklearn.ensemble import RandomForestClassifier
    rf = RandomForestClassifier()
    ##KNN
    from sklearn.neighbors import KNeighborsClassifier
    knn = KNeighborsClassifier(5)
    ##Bernoulli Naive bayes
    from sklearn.naive_bayes import BernoulliNB
    bnb = sklearn.naive_bayes.BernoulliNB()
    ##Gaussian Naive Bayes
    from sklearn.naive_bayes import GaussianNB
    gnb = GaussianNB()
    ##Decision tree
    from sklearn.tree import DecisionTreeClassifier
    dt = sklearn.tree.DecisionTreeClassifier()
    ##LogisticRegression
    from sklearn.linear_model import LogisticRegression
    lr = LogisticRegression()
    ##SVC
    from sklearn.svm import SVC
    svc = SVC(C=100)
    ##ExtraTreeClassifier
    from sklearn.tree import ExtraTreeClassifier
    etc = ExtraTreeClassifier()
    ##NN
    from sklearn.neural_network import MLPClassifier
    nn = MLPClassifier()


    ############################################################### voting 
    # generate model
    models = tools.Models(x_src,y_src,x_tar,y_tar)
    models.add(rf,'RF')
    models.add(lr,'LR')
    models.add(svc,'SVC')

    models.sort_src(show=False)
    label_voting = voting(x_tar.shape[0],
                          models.model[0],
                          models.model[1],
                          models.model[2])

    #get residual and candidate
    x_tar_residual = []
    y_tar_residual = []
    x_tar_candidate = []
    y_tar_candidate = []
    label_candidate = []
    #residual
    r_index = [i for i in range(x_tar.shape[0]) if (label_voting[i] == -1)]
    x_tar_residual = x_tar[r_index,:]
    y_tar_residual = y_tar[r_index,:]
    label_residual = label_voting[r_index]
    #candidate
    c_index = [i for i in range(x_tar.shape[0]) if (label_voting[i] != -1)]
    x_tar_candidate = x_tar[c_index,:]
    y_tar_candidate = y_tar[c_index,:]
    label_candidate = label_voting[c_index]
    
    r = y_tar_candidate.shape[0]/y_tar.shape[0]

    ############################################################in-class transfer
    x_tar_candidate_new = []
    y_tar_candidate_new = []
    x_src_new = []
    y_src_new = []
    x_tar_candidate_old = []
    y_tar_candidate_old = []

    for i, element in enumerate(label_set):
        class_index = label_set[i]
        #source data in that class i        
        src_index_i = [i for i in range(x_src.shape[0]) if (y_src[i,0] == class_index)]
        x_src_i = x_src[src_index_i,:]
        y_src_i = y_src[src_index_i,:]
        #candidate data in that class i
        tar_index_i = [i for i in range(x_tar_candidate.shape[0]) if (label_candidate[i] == class_index)]
        x_tar_candidate_i = x_tar_candidate[tar_index_i,:]
        y_tar_candidate_i = y_tar_candidate[tar_index_i,:]
        if (x_tar_candidate_i != []):
            for j in range(x_tar_candidate_i.shape[0]):
                x_tar_candidate_old.append(x_tar_candidate_i[j])
                y_tar_candidate_old.append(y_tar_candidate_i[j])
        ##################TCA
        my_tca = tca.TCA(dim=options['dim'],kerneltype=options['kerneltype'], kernelparam=options['kernelparam'], mu=options['mu'])
        if (x_tar_candidate_i.shape[0] == 0):
            x_tar_candidate_new_i = x_tar_candidate_i
    #         x_tar_candidate_i_fake = np.array(random.sample(list(x_src_i),15)) # sample from src to generate fake candidate
            x_tar_candidate_i_fake = np.array(random.sample(list(x_tar),50)) #sample from target
            x_src_new_i, x_tar_candidate_new_i_fake, x_tar_o = my_tca.fit_transform(x_src_i, x_tar_candidate_i_fake)
        else:
            x_tar_candidate_i_fake = np.array(random.sample(list(x_tar),80)) # resample to make the class with less candidate more robust
            x_src_new_i, x_tar_candidate_new_i, x_tar_o = my_tca.fit_transform(x_src_i, x_tar_candidate_i)
        #merge
        for j in range(x_src_new_i.shape[0]):
            x_src_new.append(np.array(x_src_new_i[j]).reshape((1,options['dim'])))
            y_src_new.append(y_src_i[j])
        if (x_tar_candidate_new_i.shape[0] != 0):
            for j in range(x_tar_candidate_new_i.shape[0]):
                x_tar_candidate_new.append(np.array(x_tar_candidate_new_i[j]).reshape((1,options['dim'])))
                y_tar_candidate_new.append(y_tar_candidate_i[j])

    #change the list to array
    x_tar_candidate_old = np.array(x_tar_candidate_old)
    y_tar_candidate_old = np.array(y_tar_candidate_old)
    x_src_new = np.array(x_src_new).reshape(x_src.shape[0],options['dim'])
    y_src_new = np.array(y_src_new)
    x_tar_candidate_new = np.array(x_tar_candidate_new).reshape(x_tar_candidate.shape[0],options['dim'])
    y_tar_candidate_new = np.array(y_tar_candidate_new)

    ########################################################## second annotation
    #train model on new candidate   
    knn_1 = KNeighborsClassifier(5)
    knn_1.fit(x_src_new, y_src_new)
    predicted_label_candidate = np.array(knn_1.predict(x_tar_candidate_new)).reshape((-1,1))
    acc_candidate = accuracy_score(y_tar_candidate_new, predicted_label_candidate)

    #train model on old candidate and predict on residual
    knn_2 = KNeighborsClassifier(5)
    knn_2.fit(x_tar_candidate_old, predicted_label_candidate)
    predicted_label_residual = knn_2.predict(x_tar_residual).reshape((-1,1))
    acc_residual = accuracy_score(y_tar_residual, predicted_label_residual)

    #get prediction
    y_predict = np.vstack((predicted_label_candidate, predicted_label_residual))
    #calculate final accuracy
    acc_stl = accuracy_score(np.vstack((y_tar_candidate_new, y_tar_residual)), np.vstack((predicted_label_candidate, predicted_label_residual)))

    tools.remove_files()#for JDA
    return acc_candidate,acc_residual,acc_stl

In [83]:
x_src, y_src, x_tar, y_tar = tools.getdata(Ab,y_a ,x_b ,y_b , balance=False, z_score=False)
print(x_src.shape,y_src.shape,x_tar.shape,y_tar.shape)

(505, 22) (505, 1) (497, 22) (497, 1)


In [88]:
x_src, y_src, x_tar, y_tar = tools.getdata(Ab,y_a ,x_b ,y_b , balance=True)
a,b,c = runhtca(x_src, y_src, x_tar, y_tar)
print("%-8s"%("A-B"),
      "%-10.6f"%(a),
      "%-10.6f"%(b),
      "%-10.6f"%(c))

x_src, y_src, x_tar, y_tar = tools.getdata(Ba, y_b ,x_a ,y_a , balance=True)
a,b,c = runhtca(x_src, y_src, x_tar, y_tar)
print("%-8s"%("B-A"),
      "%-10.6f"%(a),
      "%-10.6f"%(b),
      "%-10.6f"%(c))

x_src, y_src, x_tar, y_tar = tools.getdata(Bc, y_b ,x_c ,y_c , balance=True)
a,b,c = runhtca(x_src, y_src, x_tar, y_tar)
print("%-8s"%("B-C"),
      "%-10.6f"%(a),
      "%-10.6f"%(b),
      "%-10.6f"%(c))

x_src, y_src, x_tar, y_tar = tools.getdata(Cb, y_c ,x_b ,y_b , balance=True)
a,b,c = runhtca(x_src, y_src, x_tar, y_tar)
print("%-8s"%("C-B"),
      "%-10.6f"%(a),
      "%-10.6f"%(b),
      "%-10.6f"%(c))

x_src, y_src, x_tar, y_tar = tools.getdata(Ac, y_a ,x_c ,y_c , balance=True)
a,b,c = runhtca(x_src, y_src, x_tar, y_tar)
print("%-8s"%("A-C"),
      "%-10.6f"%(a),
      "%-10.6f"%(b),
      "%-10.6f"%(c))

x_src, y_src, x_tar, y_tar = tools.getdata(Ca, y_c ,x_a ,y_a , balance=True)
a,b,c = runhtca(x_src, y_src, x_tar, y_tar)
print("%-8s"%("C-A"),
      "%-10.6f"%(a),
      "%-10.6f"%(b),
      "%-10.6f"%(c))


A-B      0.818841   0.674699   0.794769  
B-A      0.334311   0.359756   0.342574  
B-C      0.634146   0.130435   0.609705  
C-B      0.186992   0.600000   0.191147  
A-C      0.804819   0.067797   0.713080  
C-A      0.544757   0.166667   0.459406  


A-B      0.818841   0.674699   0.794769  
B-A      0.334311   0.359756   0.342574  
B-C      0.634146   0.130435   0.609705  
C-B      0.186992   0.600000   0.191147  
A-C      0.804819   0.067797   0.713080  
C-A      0.544757   0.166667   0.459406  


# HTCA mean of 10 times

## balanced unstandarized

In [79]:
seed(123)
x_src, y_src, x_tar, y_tar = tools.getdata(Ab,y_a ,x_b ,y_b , balance=True)
df = pd.DataFrame(columns=['acc_can','acc_res','acc_stl'])
T = 20
for i in range(T):
    a,b,c = runhtca(x_src, y_src, x_tar, y_tar)
    df = df.append(pd.DataFrame({'acc_can':a,'acc_res':b,'acc_stl':c}, 
                      index=[i]))
a,b,c = df.mean()
print("%-8s"%("A-B"),
      "%-10.6f"%(a),
      "%-10.6f"%(b),
      "%-10.6f"%(c))

x_src, y_src, x_tar, y_tar = tools.getdata(Ba, y_b ,x_a ,y_a , balance=True)
df = pd.DataFrame(columns=['acc_can','acc_res','acc_stl'])
for i in range(T):
    a,b,c = runhtca(x_src, y_src, x_tar, y_tar)
    df = df.append(pd.DataFrame({'acc_can':a,'acc_res':b,'acc_stl':c}, 
                      index=[i]))
a,b,c = df.mean()
print("%-8s"%("B-A"),
      "%-10.6f"%(a),
      "%-10.6f"%(b),
      "%-10.6f"%(c))

x_src, y_src, x_tar, y_tar = tools.getdata(Bc, y_b ,x_c ,y_c , balance=True)
df = pd.DataFrame(columns=['acc_can','acc_res','acc_stl'])
for i in range(T):
    a,b,c = runhtca(x_src, y_src, x_tar, y_tar)
    df = df.append(pd.DataFrame({'acc_can':a,'acc_res':b,'acc_stl':c}, 
                      index=[i]))
a,b,c = df.mean()
print("%-8s"%("B-C"),
      "%-10.6f"%(a),
      "%-10.6f"%(b),
      "%-10.6f"%(c))

x_src, y_src, x_tar, y_tar = tools.getdata(Cb, y_c ,x_b ,y_b , balance=True)
df = pd.DataFrame(columns=['acc_can','acc_res','acc_stl'])
for i in range(T):
    a,b,c = runhtca(x_src, y_src, x_tar, y_tar)
    df = df.append(pd.DataFrame({'acc_can':a,'acc_res':b,'acc_stl':c}, 
                      index=[i]))
a,b,c = df.mean()
print("%-8s"%("C-B"),
      "%-10.6f"%(a),
      "%-10.6f"%(b),
      "%-10.6f"%(c))

x_src, y_src, x_tar, y_tar = tools.getdata(Ac, y_a ,x_c ,y_c , balance=True)
df = pd.DataFrame(columns=['acc_can','acc_res','acc_stl'])
for i in range(T):
    a,b,c = runhtca(x_src, y_src, x_tar, y_tar)
    df = df.append(pd.DataFrame({'acc_can':a,'acc_res':b,'acc_stl':c}, 
                      index=[i]))
a,b,c = df.mean()
print("%-8s"%("A-C"),
      "%-10.6f"%(a),
      "%-10.6f"%(b),
      "%-10.6f"%(c))

x_src, y_src, x_tar, y_tar = tools.getdata(Ca, y_c ,x_a ,y_a , balance=True)
df = pd.DataFrame(columns=['acc_can','acc_res','acc_stl'])
for i in range(T):
    a,b,c = runhtca(x_src, y_src, x_tar, y_tar)
    df = df.append(pd.DataFrame({'acc_can':a,'acc_res':b,'acc_stl':c}, 
                      index=[i]))
a,b,c = df.mean()
print("%-8s"%("C-A"),
      "%-10.6f"%(a),
      "%-10.6f"%(b),
      "%-10.6f"%(c))


A-B      0.692922   0.674583   0.751509  
B-A      0.369874   0.310182   0.363663  
B-C      0.633927   0.125892   0.612025  
C-B      0.356224   0.152061   0.260463  
A-C      0.724072   0.050684   0.610970  
C-A      0.563469   0.204470   0.519802  


A-B      0.658881   0.758998   0.758350  
B-A      0.356341   0.295877   0.348911  
B-C      0.632276   0.122052   0.613080  
C-B      0.417704   0.263826   0.380684  
A-C      0.714880   0.071606   0.615401  
C-A      0.576425   0.354958   0.543366